# Licitações realizadas pela Universidade Federal do Paraná (2014~2020)
# Tratamento de dados

## Importando a base de dados (disponível em dados.gov)

In [120]:
import pandas as pd
from glob import glob
import locale

In [121]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [122]:
arquivos = sorted(glob('BaseDados/financeiro_licit_20*.csv'))
dados = pd.concat((pd.read_csv(arquivo,  sep = ';', encoding = 'latin-1') for arquivo in arquivos), ignore_index=True)

In [123]:
dados.head()

,Ano,Modalidade de Licitação,Número/Ano,Processo,Situação,Objeto,Natureza do Objeto,Demandante,Modalidade de Compra,Homologação (mês),Valor Estimado UFPR,Valor Estimado Outros Órgãos,Valor Contratado UFPR,Valor Contratado Outros Órgãos
0,2014,Chamada Pública,001/2014,023625/2014-85,Em Andamento,Realização de Eventos (propriedade industrial)...,NaN,AGITEC,NaN,NaN,"0,00","0,00","0,00","0,00"
1,2014,Concorrência Pública,001/2014,047629/2013-78,Fracassado,"Concessão de uso de espaço físico, cantina do ...",NaN,SEPT,NaN,NaN,"0,00","0,00","0,00","0,00"
2,2014,Concorrência Pública,002/2014,048559/2013-75,Fracassado,Concessão de uso de espaço para funcionamento ...,NaN,SACOD,Contrato,NaN,"0,00","0,00","0,00","0,00"
3,2014,Concorrência Pública,003/2014,052161/2013-33,Suspenso,Construção do Edifício de Salas do Setor de Ci...,NaN,PCU,SRP,NaN,"9.998.396,80","0,00","8.662.384,47","0,00"
4,2014,Concorrência Pública,004/2014,041629/2013-64,Suspensa,Construção - Edificação - Bloco Setor de Ciênc...,NaN,PCU,SRP,NaN,"0,00","0,00","0,00","0,00"


## Tratando os dados (limpeza e organização)

In [124]:
locale.currency(10000000)

'R$ 10000000,00'

### Alterando tipo dos valores

In [125]:
correcaoValor = list(dados.columns[10:14])

for v in correcaoValor:
    dados[v] = dados[v].apply(lambda x: str(x).replace('.',''))
    dados[v] = dados[v].apply(lambda x: str(x).replace(',','.'))
    dados[v] = dados[v].apply(lambda x: x.replace(x, '0') if x.isalpha() else x)
    dados[v].fillna(0)
    dados[v] = pd.to_numeric(dados[v])
dados[10:15]

,Ano,Modalidade de Licitação,Número/Ano,Processo,Situação,Objeto,Natureza do Objeto,Demandante,Modalidade de Compra,Homologação (mês),Valor Estimado UFPR,Valor Estimado Outros Órgãos,Valor Contratado UFPR,Valor Contratado Outros Órgãos
10,2014,Concorrência Pública,010/2014,029550/2013-65,Aguarda Abertura,Concessão de espaço físico situado nas dependê...,NaN,SACOD,NaN,NaN,0.00,0.0,0.0,0.0
11,2014,Concorrência Pública,011/2014,025998/2014-91,Fase interna,Concessão de espaço físico situado no Setor Pa...,NaN,Palotina,NaN,NaN,0.00,0.0,0.0,0.0
12,2014,Pregão Eletrônico,018/2014,01.392/2014-60,Homologado,Aquisição de materiais hidráulicos para manute...,material,PCU,NaN,NaN,286235.50,0.0,280600.2,0.0
13,2014,Pregão Eletrônico,002/2014,54.855/2013-13,Homologado,Contratação de serviços de fornecimento de ref...,Serviço,RU,Contrato,NaN,1753056.00,0.0,1731024.0,0.0
14,2014,Pregão Eletrônico,003/2014,49.208/2013-81,Homologado,Aquisição de embalagens de plástico,Material,RU,SRP,NaN,62902.02,0.0,42631.0,0.0


### Retirando os valores nulos

In [126]:
si = 'Sem Informação'
dados.fillna({'Natureza do Objeto': si, 'Modalidade de Compra': si, 'Valor Estimado UFPR': 0}, inplace = True)

In [127]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360 entries, 0 to 1359
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Ano                             1360 non-null   int64  
 1   Modalidade de Licitação         1360 non-null   object 
 2   Número/Ano                      1360 non-null   object 
 3   Processo                        1360 non-null   object 
 4   Situação                        1360 non-null   object 
 5   Objeto                          1360 non-null   object 
 6   Natureza do Objeto              1360 non-null   object 
 7   Demandante                      1360 non-null   object 
 8   Modalidade de Compra            1360 non-null   object 
 9   Homologação (mês)               816 non-null    object 
 10  Valor Estimado UFPR             1360 non-null   float64
 11  Valor Estimado Outros Órgãos    1360 non-null   float64
 12  Valor Contratado UFPR           13

### Uniformizando coluna "Situação"

In [128]:
sit = dados['Situação'].value_counts()
sit

Homologado                     1036
Fase Interna                     76
Fracassado                       50
Deserto                          36
Verificação de conformidade      33
Cancelado                        25
Em Andamento                     13
Revogado                         13
Suspenso                          8
homologado                        8
Fase interna                      8
Aguarda Abertura                  7
Suspensa                          7
Anulado                           6
SUSPENSO                          6
Elaboração do Edital              5
Cancelada                         5
Aceitação                         4
IRP                               3
Deserta                           3
Parecer Jurídico                  2
deserto                           2
Anulada                           2
Fracassada                        1
cancelado na adjudicação          1
Name: Situação, dtype: int64

In [129]:
situacoes = ['Anulado', 'Suspenso', 'Fase Interna', 'Deserto', 'Homologado', 'Fracassado', 'Cancelado']
correcao = [['Anulada', 'anulada', 'anulado'], ['Suspensa', 'suspensa', 'suspenso', 'SUSPENSO'],
            ['Fase interna'], ['Deserta', 'deserto'], ['Homologada', 'homologado', 'homologada'],
            ['Fracassada'], ['cancelado na adjudicação', 'Cancelada']]

for z in range(len(situacoes)):
    dados['Situação'] = dados['Situação'].apply(lambda x: situacoes[z] if x in correcao[z] else x)

In [130]:
sit = dados['Situação'].value_counts()
sit

Homologado                     1044
Fase Interna                     84
Fracassado                       51
Deserto                          41
Verificação de conformidade      33
Cancelado                        31
Suspenso                         21
Revogado                         13
Em Andamento                     13
Anulado                           8
Aguarda Abertura                  7
Elaboração do Edital              5
Aceitação                         4
IRP                               3
Parecer Jurídico                  2
Name: Situação, dtype: int64

### Alterando nomes das colunas

In [131]:
dados.columns

Index(['Ano', 'Modalidade de Licitação', 'Número/Ano', 'Processo', 'Situação',
       'Objeto', 'Natureza do Objeto', 'Demandante', 'Modalidade de Compra',
       'Homologação (mês)', 'Valor Estimado UFPR',
       'Valor Estimado Outros Órgãos', 'Valor Contratado UFPR',
       'Valor Contratado Outros Órgãos'],
      dtype='object')

In [132]:
dados.rename(columns = {'Ano' : 'ano', 
                'Modalidade de Licitação' : 'modalidade',
                'Número/Ano' : 'numero_ano',
                'Processo' : 'processo',
                'Situação' : 'situacao',
                'Objeto' : 'objeto',
                'Natureza do Objeto' : 'natureza',
                'Demandante' : 'demandante',
                'Modalidade de Compra' : 'modalidade_compra',
                'Homologação (mês)' : 'homologacao_mes',
                'Valor Estimado UFPR' : 'valor_estimado_ufpr',
                'Valor Estimado Outros Órgãos' : 'valor_estimado_outros',
                'Valor Contratado UFPR' : 'valor_contratado_ufpr',
                'Valor Contratado Outros Órgãos': 'valor_contratado_outros'}, inplace=True)

In [133]:
dados.head()

,ano,modalidade,numero_ano,processo,situacao,objeto,natureza,demandante,modalidade_compra,homologacao_mes,valor_estimado_ufpr,valor_estimado_outros,valor_contratado_ufpr,valor_contratado_outros
0,2014,Chamada Pública,001/2014,023625/2014-85,Em Andamento,Realização de Eventos (propriedade industrial)...,Sem Informação,AGITEC,Sem Informação,NaN,0.0,0.0,0.00,0.0
1,2014,Concorrência Pública,001/2014,047629/2013-78,Fracassado,"Concessão de uso de espaço físico, cantina do ...",Sem Informação,SEPT,Sem Informação,NaN,0.0,0.0,0.00,0.0
2,2014,Concorrência Pública,002/2014,048559/2013-75,Fracassado,Concessão de uso de espaço para funcionamento ...,Sem Informação,SACOD,Contrato,NaN,0.0,0.0,0.00,0.0
3,2014,Concorrência Pública,003/2014,052161/2013-33,Suspenso,Construção do Edifício de Salas do Setor de Ci...,Sem Informação,PCU,SRP,NaN,9998396.8,0.0,8662384.47,0.0
4,2014,Concorrência Pública,004/2014,041629/2013-64,Suspenso,Construção - Edificação - Bloco Setor de Ciênc...,Sem Informação,PCU,SRP,NaN,0.0,0.0,0.00,0.0


### Retirando homologações de 2021 e alterando tipo data

In [134]:
dados.homologacao_mes = dados.homologacao_mes.str.capitalize()
dados.homologacao_mes.unique()

array([nan, 'Março', 'Fevereiro', 'Abril', 'Maio', 'Junho', 'Agosto',
       'Julho', 'Setembro', 'Novembro', 'Outubro', 'Dezembro', 'Janeiro',
       'Janeiro / 21', 'Fevereiro - 2021', 'Março de 2021',
       'Janeiro /2021'], dtype=object)

In [135]:
for x in dados.homologacao_mes:
    if '21' in str(x):
        i = dados[dados.homologacao_mes == x].index
        dados.drop(i, inplace=True)

In [136]:
dados.homologacao_mes.unique()

array([nan, 'Março', 'Fevereiro', 'Abril', 'Maio', 'Junho', 'Agosto',
       'Julho', 'Setembro', 'Novembro', 'Outubro', 'Dezembro', 'Janeiro'],
      dtype=object)

In [137]:
meses = {'Março':3, 'Fevereiro':2, 'Abril':4, 'Maio':5, 'Junho':6, 'Agosto':8,
       'Julho':7, 'Setembro':9, 'Novembro':11, 'Outubro':10, 'Dezembro':12, 'Janeiro':1}
dados.homologacao_mes = dados.homologacao_mes.map(meses).convert_dtypes(convert_integer=True)

In [138]:
dados['homologacao_mes_ano'] = dados['homologacao_mes'].astype(str) + '/' + dados['ano'].astype(str)

### Uniformizando a coluna natureza

In [141]:
natureza = ['Materiais', 'Serviço', 'Materiais e Serviço']
correcao = [['material', 'materiais', 'Material ', 'Material'],
            ['serviço', 'Serviço ', 'Serviços '],
            ['Serviço e Material', 'material e serviço', 'material/serviço', 'Material/Serviços', 'Material/Serviço', 'Serviço/material']]

for z in range(len(natureza)):
    dados.natureza = dados.natureza.apply(lambda x: natureza[z] if x in correcao[z] else x)

In [142]:
dados.natureza.unique()

array(['Sem Informação', 'Materiais', 'Serviço', 'Materiais e Serviço',
       'Material e Mão de Obra', 'Equipamento'], dtype=object)

### Reindexando os registros

In [143]:
dados.index = range(0,len(dados))

In [144]:
dados

,ano,modalidade,numero_ano,processo,situacao,objeto,natureza,demandante,modalidade_compra,homologacao_mes,valor_estimado_ufpr,valor_estimado_outros,valor_contratado_ufpr,valor_contratado_outros,homologacao_mes_ano
0,2014,Chamada Pública,001/2014,023625/2014-85,Em Andamento,Realização de Eventos (propriedade industrial)...,Sem Informação,AGITEC,Sem Informação,<NA>,0.00,0.0,0.00,0.0,NaT
1,2014,Concorrência Pública,001/2014,047629/2013-78,Fracassado,"Concessão de uso de espaço físico, cantina do ...",Sem Informação,SEPT,Sem Informação,<NA>,0.00,0.0,0.00,0.0,NaT
2,2014,Concorrência Pública,002/2014,048559/2013-75,Fracassado,Concessão de uso de espaço para funcionamento ...,Sem Informação,SACOD,Contrato,<NA>,0.00,0.0,0.00,0.0,NaT
3,2014,Concorrência Pública,003/2014,052161/2013-33,Suspenso,Construção do Edifício de Salas do Setor de Ci...,Sem Informação,PCU,SRP,<NA>,9998396.80,0.0,8662384.47,0.0,NaT
4,2014,Concorrência Pública,004/2014,041629/2013-64,Suspenso,Construção - Edificação - Bloco Setor de Ciênc...,Sem Informação,PCU,SRP,<NA>,0.00,0.0,0.00,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,2020,Regime Diferenciado de Contratação,011/2020,006361/2019-18,Homologado,Contratação de empresa de arquitetura ou engen...,Sem Informação,SUINFRA,Sem Informação,<NA>,385440.33,0.0,0.00,0.0,NaT
1351,2020,Regime Diferenciado de Contratação,012/2020,032662/2019-99,Homologado,Contratação de empresa de engenharia para a ex...,Sem Informação,SUINFRA,Sem Informação,12,92399.76,0.0,83159.78,0.0,2020-12-01
1352,2020,Regime Diferenciado de Contratação,013/2020,044949/2018-81,Homologado,Contratação de empresa de arquitetura ou engen...,Sem Informação,SUINFRA,Sem Informação,12,1938827.90,0.0,1839462.97,0.0,2020-12-01
1353,2020,Regime Diferenciado de Contratação,014/2020,053769/2020-12,Homologado,Contratação de empresa de arquitetura e/ou eng...,Sem Informação,SUINFRA,Sem Informação,12,457012.05,0.0,387999.57,0.0,2020-12-01


## Exportando dados tratados

In [145]:
dados.to_csv('dadosTratados.csv', sep = ';', index = False)